In [5]:
import random

def gcd(a, b):
    if (b == 0):
        return a
    else:
        return gcd(b, a % b)

def xgcd(a, b):
    x, old_x = 0, 1
    y, old_y = 1, 0

    while (b != 0):
        quotient = a // b
        a, b = b, a - quotient * b
        old_x, x = x, old_x - quotient * x
        old_y, y = y, old_y - quotient * y

    return a, old_x, old_y

def chooseE(totient):
    while (True):
        e = random.randrange(2, totient)

        if (gcd(e, totient) == 1):
            return e

def generate_key(key_name):

    # 在质数不太小也不太大的行范围内选择两个随机数
    rand1 = random.randint(0, 200)
    rand2 = random.randint(0, 200)

    lines = ['541','547','557','563','569','571','577','587','593','599','601','607','613','617','619','631','641','643','647','653','659','661','673','677','683','691','701','709','719','727','733','739','743','751','757','761','769','773','787','797','809','811','821','823','827','829','839','853','857','859','863','877','881','883','887','907','911','919','929','937','941','947','953','967','971','977','983','991','997','1009','1013','1019','1021','1031','1033','1039','1049','1051','1061','1063','1069','1087','1091','1093','1097','1103','1109','1117','1123','1129','1151','1153','1163','1171','1181','1187','1193','1201','1213','1217','1223','1229','1231','1237','1249','1259','1277','1279','1283','1289','1291','1297','1301','1303','1307','1319','1321','1327','1361','1367','1373','1381','1399','1409','1423','1427','1429','1433','1439','1447','1451','1453','1459','1471','1481','1483','1487','1489','1493','1499','1511','1523','1531','1543','1549','1553','1559','1567','1571','1579','1583','1597','1601','1607','1609','1613','1619','1621','1627','1637','1657','1663','1667','1669','1693','1697','1699','1709','1721','1723','1733','1741','1747','1753','1759','1777','1783','1787','1789','1801','1811','1823','1831','1847','1861','1867','1871','1873','1877','1879','1889','1901','1907','1913','1931','1933','1949','1951','1973','1979','1987','1993','1997','1999','2003','2011','2017','2027','2029','2039','2053','2063','2069','2081','2083','2087','2089','2099','2111','2113','2129','2131','2137','2141','2143','2153','2161','2179','2203','2207','2213','2221','2237','2239','2243']


    # 将质数存储在变量中
    prime1 = int(lines[rand1])
    prime2 = int(lines[rand2])

    # 计算 n, totient, e
    n = prime1 * prime2
    totient = (prime1 - 1) * (prime2 - 1)
    e = chooseE(totient)

    # c计算 d, 1 < d < totient 使得 ed = 1 (mod totient) 
    # e 和 d 是倒数 (mod totient)  
    gcd, x, y = xgcd(e, totient)

    # 确保 d 为正
    if (x < 0):
        d = x + totient
    else:
        d = x

    # 将公钥 n 和 e 写入文件
    f_public = open('public' + key_name + '.txt', 'w')
    f_public.write(str(n) + '\n')
    f_public.write(str(e) + '\n')
    f_public.close()

    f_private = open('private' + key_name + '.txt', 'w')
    f_private.write(str(n) + '\n')
    f_private.write(str(d) + '\n')
    f_private.close()

def encrypt(message, file_name = 'public_keys.txt', block_size = 2):

    try:
        fo = open(file_name, 'r')

    # 检查用户尝试使用未找到的公钥加密某些内容的可能性
    except FileNotFoundError:
        print('That file is not found.')
    else:
        n = int(fo.readline())
        e = int(fo.readline())
        fo.close()

        encrypted_blocks = []
        ciphertext = -1

        if (len(message) > 0):
            # 将密文初始化为消息第一个字符的 ASCII
            ciphertext = ord(message[0])

        for i in range(1, len(message)):
            # 如果达到最大块大小，则将密文添加到列表中
            # 重置密文，以便我们可以继续添加 ASCII 代码
            if (i % block_size == 0):
                encrypted_blocks.append(ciphertext)
                ciphertext = 0

            # 乘以 1000 将数字向左移动 3 位
            # 因为 ASCII 码最多 3 位十进制数l
            ciphertext = ciphertext * 1000 + ord(message[i])

        # 将最后一个块添加到列表中
        encrypted_blocks.append(ciphertext)

        # 通过将所有数字取为 e 的幂来加密所有数字，并将其修改为 n
        for i in range(len(encrypted_blocks)):
            encrypted_blocks[i] = str((encrypted_blocks[i]**e) % n)

        # 用这些数字创建一个字符串
        encrypted_message = " ".join(encrypted_blocks)

        return encrypted_message

def decrypt(decrypt_key_path, blocks, block_size = 2):

    fo = open(decrypt_key_path, 'r')
    n = int(fo.readline())
    d = int(fo.readline())
    fo.close()

    # 将字符串转换为整数列表
    list_blocks = blocks.split(' ')
    int_blocks = []

    for s in list_blocks:
        int_blocks.append(int(s))

    message = ""

    # 默认将列表中的每个 int 转换为 block_size 字符数，每个 int 代表两个字符
    for i in range(len(int_blocks)):
        # 通过将其取为 d 的幂并将其修改为 n 来解密所有数字
        int_blocks[i] = (int_blocks[i]**d) % n
        
        tmp = ""
        # 将每个块分解为每个字符的 ASCII 码并将其存储在消息字符串中
        for c in range(block_size):
            tmp = chr(int_blocks[i] % 1000) + tmp
            int_blocks[i] //= 1000
        message += tmp

    return message



# RSA 加密
def rsa_encryp( encryp_key_path, plaintext):
    print('正在解密......')
    print()
    cipher_text = encrypt(plaintext, encryp_key_path)
    
    print('========================密文========================')
    print(cipher_text)
    with open('ciphertext.txt', 'w') as f:
        f.write(cipher_text)
    print('============保存到 ciphertext.txt 中================')
    print()
    return cipher_text

# RSA 解密
def rsa_decrypt( decrypt_key_path, ciphertext):
    result = decrypt(decrypt_key_path, ciphertext)

    print('===================== 解密结果 =====================')
    print(result)
    with open('result.txt', 'w') as f:
        f.write(result)
    print('==============保存到 result.txt 中==================')
    print()
    return result


def get_plaintext(path):
    message = ''
    with open(path, 'r') as f:
        message =  f.read()
    
    print('========================明文========================')
    print(message)
    print('===============从 {} 中加载=============='.format(path))
    print()
    return message


if __name__ == "__main__":
    generate_key('key1')
    generate_key('key2')

    message = get_plaintext('plaintext.txt')

    # 采用公钥1进行加密，而后采用私钥1进行解密
    encryp_key_path, decrypt_key_path = 'publickey1.txt', 'privatekey1.txt'

    # 采用私钥1进行加密，而后采用公钥1进行解密
    #encryp_key_path, decrypt_key_path = 'privatekey1.txt', 'publickey1.txt'

    # 采用私钥1进行加密，而后采用公钥2进行解密
    #encryp_key_path, decrypt_key_path = 'publickey2.txt', 'publickey2.txt'

    # 采用公钥1进行加密，而后采用私钥2进行解密
    #encryp_key_path, decrypt_key_path = 'publickey1.txt', 'privatekey2.txt'

    # 采用私钥1进行加密，而后采用私钥2进行解密
    #encryp_key_path, decrypt_key_path = 'privatekey1.txt', 'privatekey2.txt'

    cipher_text = ''

    cipher_text = rsa_encryp(encryp_key_path, message)

    rsa_decrypt(decrypt_key_path, cipher_text)


========================明文========================
21200211047xyz
===============从 plaintext.txt 中加载==============

正在解密......

========================密文========================
149943 101976 298447 526643 37915 31388 37464
============保存到 ciphertext.txt 中================

===================== 解密结果 =====================
21200211047xyz
==============保存到 result.txt 中==================

